# Calculating The Stress-Energy Tensor For a Metric

This notebook demonstrates how to compute the stress-energy tensor from a spacetime metric using WarpFactory. The stress-energy tensor describes the density and flux of energy and momentum in spacetime, and is computed from the metric via Einstein's field equations:

$$T^{\mu\nu} = \frac{c^4}{8\pi G} G^{\mu\nu}$$

where $G^{\mu\nu}$ is the Einstein tensor.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from warpfactory.metrics.alcubierre import get_alcubierre_metric
from warpfactory.solver.energy import get_energy_tensor

## Create an Alcubierre Metric

First, we'll create an Alcubierre warp drive metric with the following parameters:
- Grid size: 1x20x20x20 (time, x, y, z)
- Velocity: 0.9c (90% of the speed of light)
- Bubble radius: R = 5
- Thickness parameter: sigma = 0.5

In [ ]:
# Set up grid parameters
grid_size = [1, 20, 20, 20]
world_center = [(grid_size[i] + 1) / 2 for i in range(4)]
velocity = 0.9
R = 5
sigma = 0.5

# Create the Alcubierre metric
metric = get_alcubierre_metric(
    grid_size=grid_size,
    world_center=world_center,
    velocity=velocity,
    radius=R,
    sigma=sigma
)

print(f"Metric created: {metric.name}")
print(f"Tensor type: {metric.tensor_type}")
print(f"Index type: {metric.index}")
print(f"Grid shape: {metric.tensor[(0,0)].shape}")

## Compute Energy Tensor

The stress-energy-momentum tensor can be calculated simply by calling `get_energy_tensor(metric)`. This function:
1. Computes the Christoffel symbols from the metric
2. Calculates the Ricci tensor
3. Computes the Ricci scalar
4. Builds the Einstein tensor
5. Converts to the stress-energy tensor using Einstein's field equations

In [ ]:
# Compute energy tensor
energy_tensor = get_energy_tensor(metric)

print(f"Energy tensor created: {energy_tensor.name}")
print(f"Tensor type: {energy_tensor.tensor_type}")
print(f"Index type: {energy_tensor.index}")

## Basic Plotting

Let's visualize both the metric and energy tensor components in the z-plane through the center of the bubble.

In [ ]:
# Get the central z-slice
z_slice = int(world_center[3])
t_slice = 0

# Plot metric components
fig, axes = plt.subplots(4, 4, figsize=(16, 16))
fig.suptitle(f'{metric.name} Metric Components', fontsize=16)

for i in range(4):
    for j in range(4):
        ax = axes[i, j]
        data = metric.tensor[(i, j)][t_slice, :, :, z_slice]
        
        # Plot
        im = ax.imshow(data.T, origin='lower', cmap='RdBu_r', aspect='auto')
        ax.set_title(f'g[{i},{j}]')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

In [ ]:
# Plot energy tensor components
fig, axes = plt.subplots(4, 4, figsize=(16, 16))
fig.suptitle(f'{metric.name} Energy Tensor Components', fontsize=16)

for i in range(4):
    for j in range(4):
        ax = axes[i, j]
        data = energy_tensor.tensor[(i, j)][t_slice, :, :, z_slice]
        
        # Plot
        im = ax.imshow(data.T, origin='lower', cmap='RdBu_r', aspect='auto')
        ax.set_title(f'T[{i},{j}]')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

## Examine Energy Tensor Properties

Let's examine some key properties of the energy tensor:
- Energy density: $T^{00}$ (the time-time component)
- Momentum density: $T^{0i}$ (the time-space components)
- Stress tensor: $T^{ij}$ (the space-space components)

In [ ]:
# Extract key components
energy_density = energy_tensor.tensor[(0, 0)][t_slice, :, :, z_slice]

# Plot energy density with more detail
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 2D plot
ax = axes[0]
im = ax.imshow(energy_density.T, origin='lower', cmap='RdBu_r', aspect='auto')
ax.set_title(r'Energy Density $T^{00}$')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.colorbar(im, ax=ax, label='Energy Density')

# Cross-section through center
ax = axes[1]
y_center = int(world_center[2])
ax.plot(energy_density[:, y_center])
ax.set_title(r'Energy Density Cross-Section at y=center')
ax.set_xlabel('x index')
ax.set_ylabel(r'$T^{00}$')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print statistics
print(f"Energy density statistics:")
print(f"  Min: {np.min(energy_density):.6e}")
print(f"  Max: {np.max(energy_density):.6e}")
print(f"  Mean: {np.mean(energy_density):.6e}")
print(f"  Std: {np.std(energy_density):.6e}")

In [ ]:
# Examine momentum density components
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Momentum Density Components', fontsize=16)

momentum_labels = ['x', 'y', 'z']
for idx, i in enumerate([1, 2, 3]):
    ax = axes[idx]
    data = energy_tensor.tensor[(0, i)][t_slice, :, :, z_slice]
    
    im = ax.imshow(data.T, origin='lower', cmap='RdBu_r', aspect='auto')
    ax.set_title(rf'$T^{{0{i}}}$ ({momentum_labels[idx]}-momentum)')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.colorbar(im, ax=ax)
    
    print(f"T^{{0{i}}} statistics: min={np.min(data):.6e}, max={np.max(data):.6e}")

plt.tight_layout()
plt.show()

## Summary

In this notebook, we:
1. Created an Alcubierre warp drive metric with specific parameters
2. Computed the stress-energy tensor using `get_energy_tensor()`
3. Visualized all 16 components of both the metric and energy tensor
4. Examined key physical properties including energy density and momentum density

The energy tensor reveals the energy and momentum requirements for maintaining the warp drive spacetime geometry. Note that negative energy densities (exotic matter) are required for the Alcubierre drive, which you can see in the energy density plots.